The dataset contains information about the Dollar Damage, Size, Location, and some information about the climate when the fire occured. Because it has information about Dollar Damage, which is what I aim to predict, this will be a supervised model.

After doing some data exploration, I found that there is for too much variance in Dollar Damage for any kind of regression model to be reliable. So instead, I have split the costs into classes, similar to how wildfire sizes are split into classes.

| Class A | Class B | Class C | Class D  | Class E |
|---------|---------|---------|----------|---------|
| < \$100  | \$100 - \$500  | \$500 - \$2,500 | $2,500 - \$10,000  | > \$10,000 |

These partitions were chosen so that each class makes up approximately one fifth of the dataset. This was done to prevent any one class from being so common that the model just predicts it every time.

With the data split into classes, a Neural Network model can be used to predict which class the fire is likely to be.

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [51]:
dataset = pd.read_csv("Dollar Damage.csv")
dataset.dropna(inplace=True)

In [52]:
targetFeatures = ["Cost Class"]
inputFeatures = ["Approximate Size (Acres)", "Approximate Latitude", "Approximate Longitude", "Average Temperature In Year In County", "Average Precipitation In Year In County"]

X = dataset[inputFeatures]
y = dataset[targetFeatures]

scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)

categories = [['A', 'B', 'C', 'D', 'E']]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(dataset["Cost Class"])

print("Pre-processed data :")
print(X)

print("Pre-processed class :")
print(y)

Pre-processed data :
      Approximate Size (Acres)  Approximate Latitude  Approximate Longitude  \
0                          0.0              0.529839               0.259252   
1                          0.0              0.621105               0.417000   
2                          0.0              0.761188               0.298177   
3                          0.0              0.591350               0.428583   
4                          0.0              1.000000               0.000000   
...                        ...                   ...                    ...   
1478                       1.0              0.210125               0.977571   
1479                       1.0              0.000000               0.903008   
1480                       1.0              0.270632               0.442858   
1481                       1.0              0.367306               0.651723   
1482                       1.0              0.520119               0.373262   

      Average Temperature In Y

Hyperparameters were chosen after applying the Grid Search Algorithm.

In [53]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

data_train, data_test, class_train, class_test = train_test_split(X, y, test_size=0.3)

mlp = MLPClassifier(solver = 'lbfgs', activation = 'logistic',
                    learning_rate_init = 0.2, batch_size = 10, hidden_layer_sizes = (14, 9), max_iter = 200)

mlp.fit(data_train, class_train)

pred = mlp.predict(data_test)

print("Accuracy : ", accuracy_score(class_test, pred))
print("Mean Square Error : ", mean_squared_error(class_test, pred))

pred

Accuracy :  0.40898876404494383
Mean Square Error :  2.0359550561797755


array([4, 2, 1, 0, 0, 1, 2, 0, 1, 2, 4, 1, 0, 2, 0, 1, 0, 0, 0, 4, 4, 1,
       2, 4, 4, 1, 0, 0, 0, 1, 4, 2, 1, 0, 0, 1, 1, 1, 4, 1, 1, 4, 1, 2,
       1, 0, 0, 0, 0, 3, 2, 1, 1, 1, 1, 1, 0, 4, 1, 4, 4, 0, 2, 2, 4, 2,
       2, 0, 1, 2, 0, 1, 4, 4, 4, 2, 1, 0, 4, 2, 4, 4, 1, 1, 4, 2, 1, 1,
       4, 4, 4, 0, 1, 1, 1, 4, 4, 2, 4, 2, 4, 4, 0, 2, 1, 0, 0, 0, 0, 1,
       4, 1, 4, 0, 4, 2, 1, 2, 4, 1, 1, 0, 0, 2, 1, 4, 0, 4, 1, 1, 0, 0,
       1, 2, 4, 1, 4, 0, 2, 4, 4, 1, 0, 4, 4, 0, 1, 2, 4, 4, 1, 1, 2, 0,
       2, 0, 4, 4, 0, 1, 0, 2, 4, 0, 1, 4, 2, 4, 1, 1, 0, 1, 1, 4, 4, 0,
       4, 0, 2, 1, 0, 4, 0, 4, 4, 1, 4, 1, 3, 1, 1, 1, 4, 1, 1, 4, 0, 1,
       2, 2, 2, 0, 1, 4, 4, 3, 4, 2, 4, 4, 1, 0, 1, 1, 4, 4, 4, 2, 0, 1,
       4, 4, 0, 4, 0, 1, 0, 4, 4, 4, 4, 2, 2, 0, 1, 1, 1, 4, 1, 2, 1, 1,
       2, 0, 4, 0, 4, 0, 4, 4, 1, 4, 0, 0, 1, 1, 4, 4, 1, 4, 4, 0, 4, 1,
       0, 1, 4, 4, 1, 1, 4, 2, 0, 1, 4, 4, 1, 2, 0, 1, 2, 4, 0, 1, 1, 4,
       4, 0, 1, 4, 2, 1, 4, 2, 4, 4, 2, 4, 4, 0, 1,

In [54]:
print("Weights of the neural network:")
for i, coef in enumerate(mlp.coefs_):
    print(f"Layer {i}:")
    print(coef)

Weights of the neural network:
Layer 0:
[[-2.27590783e+01  1.54708372e+01 -1.81317692e+01 -1.74993100e+01
   3.43390402e+01  2.03922952e+01  4.47258091e+00  3.27818899e+01
   1.87200177e+01  1.79963631e+01  1.40805601e+01  6.33258043e+00
  -2.59155721e+01  3.17195102e+00]
 [-1.98257553e+00  3.96435019e+00  1.46420756e+00 -1.25862254e+00
   2.14277403e-01  3.45138106e+00  4.61940549e+00  6.37352421e-01
   2.29754413e+00  5.78102875e-01  6.02753185e+00 -1.80051069e+00
  -5.12912778e-01  9.64877105e-01]
 [-3.08966484e+00  4.22219979e-01 -1.11863034e-01 -7.07390854e-01
  -4.46308760e-01 -5.14688834e-01 -2.60051129e+00  3.65266983e-01
   1.44687299e+00  3.63662102e-01 -4.74525946e+00 -2.67007981e+00
  -1.27968037e+00 -2.07032290e+00]
 [-3.24435621e+00  1.85955776e+00 -2.50115344e+00  4.31396013e-01
  -5.28767304e-01 -2.21263111e+00  2.00879347e+00 -4.64869704e-01
   1.41890524e+00  1.99467177e+00  1.20415595e+00 -4.41350225e-01
   2.48465428e-01 -4.06222574e+00]
 [-1.21492357e-01  2.7989633

In [55]:
print("Confusion Matrix for each label : ")

classNum = 0
for matrix in multilabel_confusion_matrix(class_test, pred):
  print(f"Class: {categories[0][classNum]}")
  classNum += 1

  tn, fp, fn, tp = matrix.ravel()
  print(f"         Actual Positive | Actual Negative")
  print(f"-------------------|-----|----------------")
  print(f"Predicted Positive | {str(tp).rjust(3)} | {str(fp).rjust(3)}")
  print(f"-------------------|-----|----------------")
  print(f"Predicted Negative | {str(fn).rjust(3)} | {str(tn).rjust(3)}")
  print(f"TP: {tp}, FP: {fp}, FN: {fn}, TN: {tn}\n")

print("Classification Report : ")
print(classification_report(class_test, pred))

Confusion Matrix for each label : 
Class: A
         Actual Positive | Actual Negative
-------------------|-----|----------------
Predicted Positive |  48 |  50
-------------------|-----|----------------
Predicted Negative |  52 | 295
TP: 48, FP: 50, FN: 52, TN: 295

Class: B
         Actual Positive | Actual Negative
-------------------|-----|----------------
Predicted Positive |  42 |  88
-------------------|-----|----------------
Predicted Negative |  52 | 263
TP: 42, FP: 88, FN: 52, TN: 263

Class: C
         Actual Positive | Actual Negative
-------------------|-----|----------------
Predicted Positive |  18 |  54
-------------------|-----|----------------
Predicted Negative |  65 | 308
TP: 18, FP: 54, FN: 65, TN: 308

Class: D
         Actual Positive | Actual Negative
-------------------|-----|----------------
Predicted Positive |   1 |   3
-------------------|-----|----------------
Predicted Negative |  77 | 364
TP: 1, FP: 3, FN: 77, TN: 364

Class: E
         Actual Positive |

In [56]:
def gridSearch():
  max_iterations = 200 * np.arange(1,3)
  hidden_layer_siz = [
      (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1),
      (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2), (19, 2), (20, 2),
      (10, 3), (11, 3), (12, 3), (13, 3), (14, 3), (15, 3), (16, 3), (17, 3), (18, 3), (19, 3), (20, 3),
      (10, 4), (11, 4), (12, 4), (13, 4), (14, 4), (15, 4), (16, 4), (17, 4), (18, 4), (19, 4), (20, 4),
      (10, 5), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (16, 5), (17, 5), (18, 5), (19, 5), (20, 5),
      (10, 6), (11, 6), (12, 6), (13, 6), (14, 6), (15, 6), (16, 6), (17, 6), (18, 6), (19, 6), (20, 6),
      (10, 7), (11, 7), (12, 7), (13, 7), (14, 7), (15, 7), (16, 7), (17, 7), (18, 7), (19, 7), (20, 7),
      (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8),
      (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9), (16, 9), (17, 9), (18, 9), (19, 9), (20, 9),
      (10, 10), (11, 10), (12, 10), (13, 10), (14, 10), (15, 10), (16, 10), (17, 10), (18, 10), (19, 10), (20, 10)
  ]

  learning_rates = 0.1 * np.arange(1, 3)

  param_grid = dict(learning_rate_init = learning_rates, hidden_layer_sizes = hidden_layer_siz, max_iter = max_iterations)
  # set model
  mlp = MLPClassifier(solver = 'lbfgs', activation = 'logistic',
                      learning_rate_init = 0.2, batch_size = 10, hidden_layer_sizes = (14, 9), max_iter = 200)

  # For Grid Search
  grid = GridSearchCV(estimator = mlp, param_grid = param_grid)

  # For Random Search
  # grid = RandomizedSearchCV(estimator = mlp, param_distributions = param_grid, n_iter = 10)

  grid.fit(X,y)

  print("Optimal Hyper-parameters : ", grid.best_params_)
  print("Optimal Accuracy : ", grid.best_score_)

#gridSearch()